## Introduction 
**We will deeply discuss how we define the cost function and implement solvers through the code**.

### Import Libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt

## Defining the Cost Function
<div style="font-size: 130%;">

We therefore define our cost function as the sum of two main components:
a **position-related** cost and a **direction-related** cost.
</div>

$
\Large
J(\mathbf{p}, \mathbf{n})
\;=\;
J^{\text{pos}}(\mathbf{p})
\;+\;
J^{\text{dir}}(\mathbf{n})
$

<div style="font-size: 130%;">

Since we assume that ground truth trajectory data is unknow, balance between data fidelity and smoothness is very important. \
So we can define our two parts of the **cost** function as also two parts:
</div>

$
\Large
J^{\text{pos}}(\mathbf{p}) \;=\; J^{\text{pos}}_{fidelety} + J^{\text{pos}}_{smoothness}
$\
$
\Large
J^{\text{dir}}(\mathbf{n}) \;=\; J^{\text{dir}}_{fidelety} + J^{\text{dir}}_{smoothness}
$


---

<div style="font-size: 130%;">

Fidelity parts are relatively more intuative to figure out. \
Since we want our cost function to penalty our predictions when it is not close to the original (noisy) data, we can directly use `Sum Square Error` or `Mean Square Error`

</div>

$
\Large
J^{\text{pos}}_{fidelety}(\mathbf{p}) \;=\;
\sum_{i=1}^{N} (\mathbf{p}_i - \mathbf{y^p}_i)^2
$\
$
\Large
J^{\text{dir}}_{fidelety}(\mathbf{n}) \;=\;
\sum_{i=1}^{N} (\mathbf{n}_i - \mathbf{y^n}_i)^2
$

---

<div style="font-size: 130%;">

To **penalize non-smoothness** in the reconstructed trajectory,  
we can examine the changes in position over discrete time steps.

The **difference** between two consecutive positions approximates the **velocity**: \
$
\Large
\mathbf{v}_i = \mathbf{p}_{i+1} - \mathbf{p}_i
$

Similarly, the **change** in velocity between two steps approximates the **acceleration**: \
$
\Large
\mathbf{a}_i = \mathbf{v}_i - \mathbf{v}_{i-1}
$

Substituting the expressions for velocity, we get: \
$
\Large
\mathbf{a}_i 
= (\mathbf{p}_{i+1} - \mathbf{p}_i) - (\mathbf{p}_i - \mathbf{p}_{i-1})
= \mathbf{p}_{i+1} - 2\mathbf{p}_i + \mathbf{p}_{i-1}
\;\approx\; \text{acceleration (rate of change of velocity)}.
$
\

Therefore, to **encourage smoothness**, we penalize large accelerations using the following regularization term: \
\
$
\Large
J^{\text{pos}}_{smoothness}(\mathbf{p}) \;=\;
\lambda_{1} \sum_{i=2}^{N-1} (\mathbf{p}_{i+1} - 2\mathbf{p}_i + \mathbf{p}_{i-1})^2
$\
\
\
Exactly the same for direction, to **encourage smoothness**, \
\
$
\Large
J^{\text{dir}}_{smoothness}(\mathbf{n}) \;=\;
\lambda_{2} \sum_{i=2}^{N-1} (\mathbf{n}_{i+1} - 2\mathbf{n}_i + \mathbf{n}_{i-1})^2
$

---

So here our **cost** function: \
$
\Large
J(\mathbf{p}, \mathbf{n}) \;=\;
\\
\sum_{i=1}^{N} (\mathbf{p}_i - \mathbf{y^p}_i)^2
\;+\;
\\
\lambda_{1} \sum_{i=2}^{N-1} (\mathbf{p}_{i+1} - 2\mathbf{p}_i + \mathbf{p}_{i-1})^2 
\;+\;
\\
\sum_{i=1}^{N} (\mathbf{n}_i - \mathbf{y^n}_i)^2
\;+\;
\\
\lambda_{2} \sum_{i=2}^{N-1} (\mathbf{n}_{i+1} - 2\mathbf{n}_i + \mathbf{n}_{i-1})^2 
$
</div>

### Read data

In [ ]:
data = np.load('drone_trajectory.npz')
trajectory = data.get('trajectory_noisy')

positions = trajectory[:, :3]  # y^p
nv        = trajectory[:, 3:6] # y^n

In [ ]:

def cost(p, n, y_p, y_n, lam_1, lam_2):
    # position fidelity
    fidelity_p = np.sum(np.linalg.norm(p - y_p, axis=1)**2)

    # position smoothness (second difference)
    smooth_p = np.sum(
        np.linalg.norm(p[2:] - 2*p[1:-1] + p[:-2], axis=1)**2
    )

    # direction fidelity
    fidelity_n = np.sum(np.linalg.norm(n - y_n, axis=1)**2)

    # direction smoothness
    smooth_n = np.sum(
        np.linalg.norm(n[2:] - 2*n[1:-1] + n[:-2], axis=1)**2
    )

    return fidelity_p + lam_1 * smooth_p + fidelity_n + lam_2 * smooth_n

<div style="font-size: 130%;">

## Gradient of the Cost Function

$
\Large
\nabla J(\mathbf{p}, \mathbf{n})
\;=\;
\begin{bmatrix} \frac{\partial J(\mathbf{p}, \mathbf{n})}{\partial \mathbf{p}} \\ \frac{\partial J(\mathbf{p}, \mathbf{n})}{\partial \mathbf{n}} \end{bmatrix}
$

$
\Large
\frac{\partial J(\mathbf{p}, \mathbf{n})}{\partial \mathbf{p}}
\;=\;
\frac{\partial J^{\text{pos}}_{fidelety}(\mathbf{p})}{\partial \mathbf{p}}
\;+\;
\frac{\partial J^{\text{dir}}_{fidelety}(\mathbf{n})}{\partial \mathbf{p}}
\;+\;
\frac{\partial J^{\text{pos}}_{smoothness}(\mathbf{p})}{\partial \mathbf{p}}
\;+\;
\frac{\partial J^{\text{dir}}_{smoothness}(\mathbf{n})}{\partial \mathbf{p}}
$


$
\Large
\frac{\partial J(\mathbf{p}, \mathbf{n})}{\partial \mathbf{n}} 
\;=\;
\frac{\partial J^{\text{pos}}_{fidelety}(\mathbf{p})}{\partial \mathbf{n}}
\;+\;
\frac{\partial J^{\text{dir}}_{fidelety}(\mathbf{n})}{\partial \mathbf{n}}
\;+\;
\frac{\partial J^{\text{pos}}_{smoothness}(\mathbf{p})}{\partial \mathbf{n}}
\;+\;
\frac{\partial J^{\text{dir}}_{smoothness}(\mathbf{n})}{\partial \mathbf{n}}
$

Some of them **cancels out**

$
\Large
\frac{\partial J(\mathbf{p}, \mathbf{n})}{\partial \mathbf{p}}
\;=\;
\frac{\partial J^{\text{pos}}_{fidelety}(\mathbf{p})}{\partial \mathbf{p}}
\;+\;
\frac{\partial J^{\text{pos}}_{smoothness}(\mathbf{p})}{\partial \mathbf{p}}
$


$
\Large
\frac{\partial J(\mathbf{p}, \mathbf{n})}{\partial \mathbf{n}} 
\;=\;
\frac{\partial J^{\text{dir}}_{fidelety}(\mathbf{n})}{\partial \mathbf{n}}
\;+\;
\frac{\partial J^{\text{dir}}_{smoothness}(\mathbf{n})}{\partial \mathbf{n}}
$
</div>

In [ ]:
p = np.random.randn(positions.shape[0], positions.shape[1])
y = positions
lam = 0

c = cost(p, y, lam)
print(c)

8559.67064302868
